# Fast kSVD test using Ariel5's orthogonal mp pytorch implementation

https://github.com/Ariel5/omp-parallel-gpu-python<br>
https://github.com/nel215/ksvd/

In [2]:
# Using newly preprocessed subjects

import pickle

metadictname = '/home/anton/Documents/Tulane/Research/PNC_Good/PNC_agesexwrat.pkl'
alltsname = '/home/anton/Documents/Tulane/Research/PNC_Good/PNC_PowerTS_float2.pkl'

with open(metadictname, 'rb') as f:
    metadict = pickle.load(f)

with open(alltsname, 'rb') as f:
    allts = pickle.load(f)
    
print(list(metadict.keys()))
print(list(allts.keys()))
print('Complete')

['age', 'sex', 'wrat', 'missingage', 'missingsex', 'missingwrat', 'failedqc']
['emoid', 'nback', 'rest']
Complete


In [3]:
# Using newly preprocessed subjects

import pickle

metadictname = '/home/anton/Documents/Tulane/Research/PNC_Good/PNC_agesexwrat.pkl'
alltsname = '/home/anton/Documents/Tulane/Research/PNC_Good/PNC_PowerTS_float2.pkl'

with open(metadictname, 'rb') as f:
    metadict = pickle.load(f)

with open(alltsname, 'rb') as f:
    allts = pickle.load(f)
    
print(list(metadict.keys()))
print(list(allts.keys()))
print('Complete')

['age', 'sex', 'wrat', 'missingage', 'missingsex', 'missingwrat', 'failedqc']
['emoid', 'nback', 'rest']
Complete


In [4]:
'''
Get subjects that have all tasks and paras specified
Functions for creating independent and response variables
'''

import numpy as np

def get_subs(allts, metadict, tasks, paras):
    # Get subs for all paras
    for i,para in enumerate(paras):
        tmpset = set([int(sub[4:]) for sub in allts[para].keys()])
        if i == 0:
            paraset = tmpset
        else:
            paraset = paraset.intersection(tmpset)
    # Get subs for all tasks
    for i,task in enumerate(tasks):
        tmpset = set([sub for sub in metadict[task].keys()])
        if i == 0:
            taskset = tmpset
        else:
            taskset = paraset.intersection(tmpset)
    # Remove QC failures
    allsubs = taskset.intersection(paraset)
    for badsub in metadict['failedqc']:
        try:
            allsubs.remove(int(badsub[4:]))
        except:
            pass
    return allsubs

def get_X(allts, paras, subs):
    X = []
    for para in paras:
        pX = [allts[para][f'sub-{sub}'] for sub in subs]
        pX = np.stack(pX)
        X.append(pX)
    return X

def get_y(metadict, tasks, subs):
    y = []
    for task in tasks:
        if task == 'age' or task == 'wrat':
            var = [metadict[task][sub] for sub in subs]
            var = np.array(var)
            y.append(var)
        if task == 'sex':
            maleness = [metadict[task][sub] == 'M' for sub in subs]
            maleness = np.array(maleness)
            sex = np.stack([maleness, 1-maleness], axis=1)
            y.append(sex)
    return y

subs = get_subs(allts, metadict, ['age'], ['rest', 'nback', 'emoid'])
print(len(subs))

X = get_X(allts, ['rest', 'nback', 'emoid'], subs)
print(X[0].shape)

847
(847, 264, 124)


In [5]:
# TS to condensed FC

from scipy import signal

def butter_bandpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = [cutoff[0] / nyq, cutoff[1] / nyq]
    b, a = signal.butter(order, normal_cutoff, btype='band', analog=False)
    return b, a

def butter_bandpass_filter(data, cutoff, fs, order=5):
    b, a = butter_bandpass(cutoff, fs, order=order)
    y = signal.filtfilt(b, a, data)
    return y

tr = 1.83

def filter_design_ts(X):
    Xs = []
    for i in range(X.shape[0]):
        nX = butter_bandpass_filter(X[i], [0.01, 0.2], 1/tr)
        Xs.append(nX)
    return np.stack(Xs)

def ts_to_flat_fc(X):
    p = np.corrcoef(X)
    a,b = np.triu_indices(p[0].shape[0], 1)
    p = p[a,b]
    return p

p = [np.stack([ts_to_flat_fc(ts) for ts in filter_design_ts(Xp)]) for Xp in X]
print(p[0].shape)

(847, 34716)


In [8]:
# coding:utf-8
import numpy as np
import scipy as sp
from sklearn.linear_model import orthogonal_mp_gram
import torch

class ApproximateKSVD(object):
    def __init__(self, n_components, max_iter=10, tol=1e-6,
                 transform_n_nonzero_coefs=None):
        """
        Parameters
        ----------
        n_components:
            Number of dictionary elements

        max_iter:
            Maximum number of iterations

        tol:
            tolerance for error

        transform_n_nonzero_coefs:
            Number of nonzero coefficients to target
        """
        self.components_ = None
        self.max_iter = max_iter
        self.tol = tol
        self.n_components = n_components
        self.transform_n_nonzero_coefs = transform_n_nonzero_coefs

    def _update_dict(self, X, D, gamma):
        for j in range(self.n_components):
            I = gamma[:, j] > 0
            if np.sum(I) == 0:
                continue

            D[j, :] = 0
            g = gamma[I, j].T
            r = X[I, :] - gamma[I, :].dot(D)
            d = r.T.dot(g)
            d /= np.linalg.norm(d)
            g = r.dot(d)
            D[j, :] = d
            gamma[I, j] = g.T
        return D, gamma

    def _initialize(self, X):
        if min(X.shape) < self.n_components:
            D = np.random.randn(self.n_components, X.shape[1])
        else:
            X = torch.from_numpy(X).float().cuda()
            u, s, vt = torch.linalg.svd(X, full_matrices=False)
            u, s, vt = [mat.detach().cpu().numpy() for mat in [u, s, vt]]
#           #sp.sparse.linalg.svds(X, k=self.n_components)
            D = np.dot(np.diag(s), vt)[:self.n_components,:]
        D /= np.linalg.norm(D, axis=1)[:, np.newaxis]
        return D

    def _transform(self, D, X):
        gram = D.dot(D.T)
        Xy = D.dot(X.T)

        n_nonzero_coefs = self.transform_n_nonzero_coefs
        if n_nonzero_coefs is None:
            n_nonzero_coefs = int(0.1 * X.shape[1])

        return orthogonal_mp_gram(
            gram, Xy, n_nonzero_coefs=n_nonzero_coefs).T

    def fit(self, X):
        """
        Parameters
        ----------
        X: shape = [n_samples, n_features]
        """
        D = self._initialize(X)
        for i in range(self.max_iter):
            print(f'Start iter {i}')
            gamma = self._transform(D, X)
            e = np.linalg.norm(X - gamma.dot(D))
            if e < self.tol:
                break
            D, gamma = self._update_dict(X, D, gamma)

        self.components_ = D
        return self

    def transform(self, X):
        return self._transform(self.components_, X)

print('Complete')

Complete


In [9]:
import time

ncodes = 800
nnzero = 100
ntrain = max(ncodes, 400)

idcs = np.arange(p[0].shape[0])
np.random.shuffle(idcs)

t0 = time.time()
ksvd = ApproximateKSVD(n_components=ncodes, transform_n_nonzero_coefs=nnzero, max_iter=1)
ksvd.fit(p[0][idcs[0:ntrain]])
z = ksvd.transform(p[0])
t1 = time.time()
print(f'{t1-t0} seconds')
print(z.shape)

Start iter 0
397.67435693740845 seconds
(847, 800)


In [10]:
import torch

nreps = 10
trainsizes = [30,50,100,200,300,400,500,600,700,800]
res = np.zeros((nreps,len(trainsizes)))
l2 = 1e3

for rep in range(nreps):
    losses = []

    idcs = np.arange(z.shape[0])
    np.random.shuffle(idcs)

    for ntrain in trainsizes:
        xps = torch.from_numpy(np.asarray(z)).float().cuda()
        xps = torch.cat([xps, torch.ones(xps.shape[0], 1).float().cuda()], dim=1)
        xtr = xps[idcs[:ntrain]]
        xt = xps[idcs[ntrain:]]

        y = get_y(metadict, ['age'], subs)[0]
        y_t = torch.from_numpy(y).float().cuda()
        ytr = y_t[idcs[:ntrain]]
        yt = y_t[idcs[ntrain:]]

        # REDUCE THIS TO GET GOOD RESULTS WITH SPARSITY 0.01->0.001 or 0.0001
        w, _, _, _ = torch.linalg.lstsq(xtr.T@xtr + l2*torch.eye(ncodes+1).float().cuda(), xtr.T@ytr)
#         w, _, _, _ = torch.linalg.lstsq(xtr, ytr)

        print(torch.mean((yt-xt@w)**2)**0.5)
        losses.append(float(torch.mean((yt-xt@w)**2)**0.5))
            
    print(f'Finished {rep}')
    res[rep,:] = losses
    
print(np.mean(res, axis=0))
print(np.std(res, axis=0))

tensor(48.3423, device='cuda:0')
tensor(44.0789, device='cuda:0')
tensor(38.8934, device='cuda:0')
tensor(35.2361, device='cuda:0')
tensor(34.5684, device='cuda:0')
tensor(34.1900, device='cuda:0')
tensor(34.1446, device='cuda:0')
tensor(31.8684, device='cuda:0')
tensor(31.6421, device='cuda:0')
tensor(29.8326, device='cuda:0')
Finished 0
tensor(45.3498, device='cuda:0')
tensor(41.4417, device='cuda:0')
tensor(38.2623, device='cuda:0')
tensor(35.8105, device='cuda:0')
tensor(33.4743, device='cuda:0')
tensor(32.7844, device='cuda:0')
tensor(33.2585, device='cuda:0')
tensor(32.3238, device='cuda:0')
tensor(33.2109, device='cuda:0')
tensor(31.4636, device='cuda:0')
Finished 1
tensor(49.3059, device='cuda:0')
tensor(44.8776, device='cuda:0')
tensor(37.0280, device='cuda:0')
tensor(36.3713, device='cuda:0')
tensor(34.9822, device='cuda:0')
tensor(34.8250, device='cuda:0')
tensor(34.4912, device='cuda:0')
tensor(33.1836, device='cuda:0')
tensor(32.3107, device='cuda:0')
tensor(30.2710, devic

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F

mseLoss = nn.MSELoss()

class MLP(nn.Module):
    def __init__(self, ncodes):
        super(MLP, self).__init__()
        self.l1 = nn.Linear(ncodes, 40).float().cuda()
        self.l2 = nn.Linear(40,1).float().cuda()
        
    def train(self, xtr, ytr, nepochs=1000, lr=1e-1, l1=1e-1, l2=1e-4, pperiod=100, verbose=False):
        optim = torch.optim.Adam(self.parameters(), lr=lr, weight_decay=l2)
        sched = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=10, factor=0.75, eps=1e-7)
        mseLoss = nn.MSELoss()
        
        for epoch in range(nepochs):
            optim.zero_grad()
            yhat = self(xtr)
            loss = mseLoss(yhat, ytr)**0.5
            l1loss = l1*torch.sum(torch.abs(self.l1.weight))
            (loss+l1loss).backward()
            optim.step()
            sched.step(loss)
            if verbose:
                if epoch % pperiod == 0 or epoch == nepochs-1:
                    print(f'{epoch} {[float(l) for l in [loss, l1loss]]} {sched._last_lr}')
                    
    def predict(self, xt, yt):
        with torch.no_grad():
            return mseLoss(self(xt), yt)**0.5
                    
    def forward(self, x):
        x = F.relu(self.l1(x))
        x = self.l2(x).squeeze()
        return x
    
nreps = 10
trainsizes = [30,50,100,200,300,400,500,600,700,800]
res = np.zeros((nreps,len(trainsizes)))

for rep in range(nreps):

    idcs = np.arange(z.shape[0])
    np.random.shuffle(idcs)

    losses = []

    for ntrain in trainsizes:
        xps = torch.from_numpy(np.asarray(z)).float().cuda()
        xtr = xps[idcs[:ntrain]]
        xt = xps[idcs[ntrain:]]

        y = get_y(metadict, ['age'], subs)[0]
        y_t = torch.from_numpy(y).float().cuda()
        ytr = y_t[idcs[:ntrain]]
        yt = y_t[idcs[ntrain:]]

        mlp = MLP(ncodes)
        # 1e-3 good for age 1e-2 good for wrat
        mlp.train(xtr, ytr, lr=1e-2, nepochs=1000, l1=1e0, l2=1e-3)
        loss = mlp.predict(xt, yt)

        losses.append(float(loss))
        print(float(loss))

    res[rep,:] = losses
    print(f'Finished {rep}')

    print(np.mean(res, axis=0))
    print(np.std(res, axis=0))

41.99235534667969
39.626373291015625
42.756378173828125


KeyboardInterrupt: 